In [1]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# TODO

In [2]:
#export
import datetime, itertools
import sklearn.metrics as skm
from tritonlytics_ai.utils import *

from fastai2 import __version__ as fa2_version
from fastai2.text.all import *

import spacy
spacy_en = spacy.load('en_core_web_sm')
spacy_es = spacy.load('es_core_news_sm')

In [3]:
#hide
import pdb, gc

# pandas and plotting config
import seaborn as sns
sns.set_style('whitegrid')

plt.rcParams['figure.figsize'] = (9,6)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

from nbdev.showdoc import *
from fastcore.test import *

In [4]:
#hide
print(f'fastai version: {fa2_version}')

fastai version: 0.0.18


In [5]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}')

Using GPU #1


## Data

In [6]:
f = json.loads((RAW_DATA_PATH/'verbatim-inference.json').read_text())

In [7]:
inf_df = pd.DataFrame(f['data'])
print(len(inf_df)); inf_df.head(2)

698


,id,survey_id,question_ans_id,answer_text,answer_text_non_english,language,survey_type_id,benchmark_survey_type,client_id,question_report_abbr,question_class,question_text,question_category_id,question_category_abbr,question_category_label,benchmark_level1,benchmark_level2,benchmark_level3,client_benchmark_level,rsp_id,rsp_language,rsp_start_date,rsp_last_activity_date,custom_value1,custom_value2,custom_value3,custom_value4,custom_value5,custom_value6,custom_value7,custom_value8,custom_value9,custom_value10,custom_value11,custom_value12,custom_value13,custom_value14,custom_value15,benchmark_identity,benchmark_sub_identity,client_identity,client_sub_identity,group_id,group_code,group_name,group_level1_code,group_level1_name,group_level2_code,group_level2_name,group_level3_code,group_level3_name,group_level4_code,group_level4_name,group_level5_code,group_level5_name,group_level6_code,group_level6_name,group_level7_code,group_level7_name,group_level8_code,group_level8_name
0,660404,396,93069,I believe ANR leadership is completely disconnected from the work of campus-based specialists. W...,None,English,47,SAW,UCANR,Comments re Work Environment at UC ANR,Verbatim-Comments,Please provide any additional feedback regarding the work environment at UC ANR. Your comments w...,1141,None,Comments,None,None,None,3,480447,English,2020-03-17T09:17:39.887,2020-03-17T09:29:23.917,ACAD,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6988,260200,UCD SPECIALISTS,999999,UC Agriculture & Natural Resources,200000,AVP Programs and Initiatives,260000.0,All CE Specialists,260200.0,UCD SPECIALISTS,NaN,None,None,None,None,None,None,None
1,660405,396,93069,I'm a new employee.,None,English,47,SAW,UCANR,Comments re Work Environment at UC ANR,Verbatim-Comments,Please provide any additional feedback regarding the work environment at UC ANR. Your comments w...,1141,None,Comments,None,None,None,3,480450,English,2020-03-17T07:49:25.93,2020-03-17T07:57:27.78,STAFF,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6971,240610,Imperial,999999,UC Agriculture & Natural Resources,200000,AVP Programs and Initiatives,240000.0,UC Cooperative Extension Offices,240600.0,UCCE South,240610.0,Imperial,None,None,None,None,None,None


For summarization, exclude "recognition" verbatims and ensure "answer_text" is typed as `str` for the aggregate operation below.

In [8]:
df = inf_df[~inf_df.question_category_label.str.contains('recognition', case=False)].copy()
df.answer_text = df.answer_text.astype(str)
df.head(2)

,id,survey_id,question_ans_id,answer_text,answer_text_non_english,language,survey_type_id,benchmark_survey_type,client_id,question_report_abbr,question_class,question_text,question_category_id,question_category_abbr,question_category_label,benchmark_level1,benchmark_level2,benchmark_level3,client_benchmark_level,rsp_id,rsp_language,rsp_start_date,rsp_last_activity_date,custom_value1,custom_value2,custom_value3,custom_value4,custom_value5,custom_value6,custom_value7,custom_value8,custom_value9,custom_value10,custom_value11,custom_value12,custom_value13,custom_value14,custom_value15,benchmark_identity,benchmark_sub_identity,client_identity,client_sub_identity,group_id,group_code,group_name,group_level1_code,group_level1_name,group_level2_code,group_level2_name,group_level3_code,group_level3_name,group_level4_code,group_level4_name,group_level5_code,group_level5_name,group_level6_code,group_level6_name,group_level7_code,group_level7_name,group_level8_code,group_level8_name
0,660404,396,93069,I believe ANR leadership is completely disconnected from the work of campus-based specialists. W...,None,English,47,SAW,UCANR,Comments re Work Environment at UC ANR,Verbatim-Comments,Please provide any additional feedback regarding the work environment at UC ANR. Your comments w...,1141,None,Comments,None,None,None,3,480447,English,2020-03-17T09:17:39.887,2020-03-17T09:29:23.917,ACAD,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6988,260200,UCD SPECIALISTS,999999,UC Agriculture & Natural Resources,200000,AVP Programs and Initiatives,260000.0,All CE Specialists,260200.0,UCD SPECIALISTS,NaN,None,None,None,None,None,None,None
1,660405,396,93069,I'm a new employee.,None,English,47,SAW,UCANR,Comments re Work Environment at UC ANR,Verbatim-Comments,Please provide any additional feedback regarding the work environment at UC ANR. Your comments w...,1141,None,Comments,None,None,None,3,480450,English,2020-03-17T07:49:25.93,2020-03-17T07:57:27.78,STAFF,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6971,240610,Imperial,999999,UC Agriculture & Natural Resources,200000,AVP Programs and Initiatives,240000.0,UC Cooperative Extension Offices,240600.0,UCCE South,240610.0,Imperial,None,None,None,None,None,None


Example of how we can aggregate comments using a `groupby`

In [9]:
client_grps_df = df.groupby(['benchmark_survey_type', 'client_id'], as_index = False).agg({
    'answer_text': [''.join, 'count']
})

client_grps_df.columns = ['_'.join(col).rstrip('_') for col in client_grps_df.columns.values]
print(len(client_grps_df)); client_grps_df.head()

1


,benchmark_survey_type,client_id,answer_text_join,answer_text_count
0,SAW,UCANR,I believe ANR leadership is completely disconnected from the work of campus-based specialists. W...,429


In [10]:
client_grps_df.iloc[0].answer_text_join[:1000]

'I believe ANR leadership is completely disconnected from the work of campus-based specialists. When I am responding regarding "my department" in this survey it is my campus department I am evaluating, not an ANR department.I\'m a new employee.Because salaries tend to be much lower than other opportunities in my area, and because the bureaucracy can be challenging, in general I think working UCANR is challenging even though other aspects are very rewarding.How likely to recommend working to a friend depends on the friend. If they like the business world of action, UC is bogged down in paper and protocol.  But not as bad as the federal government.  ANR is where you can see change happen, although trying to document it, is a bit overdone, Change takes time and UC wants to see it on paper right now.  Leadership is a difficult position and at times ours seems arbitrary.  We have committees and leadership should listen and act on the suggestions from those committees.The one thing that woul

Now we'll define the various labels by which we want to aggregate verbatims

In [11]:
core_lbls = ['benchmark_survey_type', 'client_id', 'question_ans_id', 'question_report_abbr']

benchmark_lbls = ['question_category_abbr', 'benchmark_level1', 'benchmark_level2', 'benchmark_level3']

rsp_lbls = [
    'custom_value1', 'custom_value2', 'custom_value3', 'custom_value4', 'custom_value5',
    'custom_value6', 'custom_value7', 'custom_value8', 'custom_value9', 'custom_value10',
    'custom_value11', 'custom_value12', 'custom_value13', 'custom_value14', 'custom_value15',
    'benchmark_identity', 'benchmark_sub_identity', 'client_identity', 'client_sub_identity'
]

grp_lvl_lbls = [
    'group_level1_code', 'group_level2_code', 'group_level3_code', 'group_level4_code',
    'group_level5_code', 'group_level6_code', 'group_level7_code', 'group_level8_code',
]

In [12]:
def build_grouped_df(lbls):
    grouped_df = df.groupby(lbls, as_index = False).agg({'answer_text': [''.join, 'count']})
    grouped_df.columns = ['_'.join(col).rstrip('_') for col in grouped_df.columns.values]
    return grouped_df

In [13]:
# for summarizing all comments
summarization_df = build_grouped_df(core_lbls)

# for summarizing by each benchmark level (1-3)
benchmark_dfs = [ build_grouped_df(core_lbls + benchmark_lbls[:idx+1]) for idx in range(len(benchmark_lbls)) ]
summarization_df = pd.concat([ summarization_df, *benchmark_dfs ], axis=0, ignore_index=True)

# for summarizing at each group level (1-8)
groups_dfs = [ build_grouped_df(core_lbls + grp_lvl_lbls[:idx+1]) for idx in range(len(grp_lvl_lbls)) ]
summarization_df = pd.concat([ summarization_df, *groups_dfs ], axis=0, ignore_index=True)

# for summarizing at each rsp custom value (1-15)
rsp_dfs = [ build_grouped_df(core_lbls + [lbl]) for lbl in rsp_lbls ]
summarization_df = pd.concat([ summarization_df, *rsp_dfs ], axis=0, ignore_index=True)

# for summarizing at each rsp custom value (1-15) at each group level (1-8)
perms = [ grp_lvl_lbls[:idx+1] + [rsp_lbl] for idx in range(len(grp_lvl_lbls)) for rsp_lbl in rsp_lbls]
grp_rsp_dfs = [ build_grouped_df(core_lbls + perm) for perm in perms ]
summarization_df = pd.concat([ summarization_df, *grp_rsp_dfs ], axis=0, ignore_index=True)

In [14]:
summarization_df.head(2)

,benchmark_survey_type,client_id,question_ans_id,question_report_abbr,answer_text_join,answer_text_count,question_category_abbr,benchmark_level1,benchmark_level2,benchmark_level3,group_level1_code,group_level2_code,group_level3_code,group_level4_code,group_level5_code,group_level6_code,group_level7_code,group_level8_code,custom_value1,custom_value2,custom_value3,custom_value4,custom_value5,custom_value6,custom_value7,custom_value8,custom_value9,custom_value10,custom_value11,custom_value12,custom_value13,custom_value14,custom_value15,benchmark_identity,benchmark_sub_identity,client_identity,client_sub_identity
0,SAW,UCANR,93005,Conduct & Behavioral - Comments,Is it possible the commitment to diversity is so strong that it goes so far in one direction tha...,136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SAW,UCANR,93069,Comments re Work Environment at UC ANR,I believe ANR leadership is completely disconnected from the work of campus-based specialists. W...,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
summarization_df.replace(to_replace='None', value=np.nan, inplace=True)
summarization_df.drop_duplicates(inplace=True)
summarization_df.reset_index(inplace=True)

In [16]:
print(len(summarization_df))
summarization_df.head()

371


,index,benchmark_survey_type,client_id,question_ans_id,question_report_abbr,answer_text_join,answer_text_count,question_category_abbr,benchmark_level1,benchmark_level2,benchmark_level3,group_level1_code,group_level2_code,group_level3_code,group_level4_code,group_level5_code,group_level6_code,group_level7_code,group_level8_code,custom_value1,custom_value2,custom_value3,custom_value4,custom_value5,custom_value6,custom_value7,custom_value8,custom_value9,custom_value10,custom_value11,custom_value12,custom_value13,custom_value14,custom_value15,benchmark_identity,benchmark_sub_identity,client_identity,client_sub_identity
0,0,SAW,UCANR,93005,Conduct & Behavioral - Comments,Is it possible the commitment to diversity is so strong that it goes so far in one direction tha...,136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,SAW,UCANR,93069,Comments re Work Environment at UC ANR,I believe ANR leadership is completely disconnected from the work of campus-based specialists. W...,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,SAW,UCANR,93005,Conduct & Behavioral - Comments,Is it possible the commitment to diversity is so strong that it goes so far in one direction tha...,136,EDI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,SAW,UCANR,93005,Conduct & Behavioral - Comments,Is it possible the commitment to diversity is so strong that it goes so far in one direction tha...,136,NaN,NaN,NaN,NaN,999999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,SAW,UCANR,93069,Comments re Work Environment at UC ANR,I believe ANR leadership is completely disconnected from the work of campus-based specialists. W...,293,NaN,NaN,NaN,NaN,999999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Modeling

We'll use BART out-of-the-box, pre-trained on cnn/daily mail and output 3 predictions per row

In [30]:
with torch.cuda.device('cuda:1'):
    torch.cuda.empty_cache()

In [31]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [33]:
hf_tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')

hf_model = BartForConditionalGeneration.from_pretrained('bart-large-cnn')
hf_model = hf_model.to('cuda:1')

hf_tokenizer2 = BartTokenizer.from_pretrained('bart-large-xsum')

hf_model2 = BartForConditionalGeneration.from_pretrained('bart-large-xsum')
hf_model2 = hf_model2.to('cuda:1')

In [34]:
%%time

summarization_res = []
for k,v in summarization_df.iterrows():
    # get preds using pretrained cnndm BART
    inputs = hf_tokenizer.batch_encode_plus([v.answer_text_join], max_length=1024, return_tensors='pt')
    
    # generate summary
    gen_text = hf_model.generate(inputs['input_ids'].to('cuda:1'), 
                                 max_length=130, min_length=30,
                                 early_stopping=True, num_beams=4, num_return_sequences=3)
    
    outputs = [ hf_tokenizer.decode(txt, skip_special_tokens=True, clean_up_tokenization_spaces=False)
               for txt in gen_text ]
    
    # get preds using pretrained xsum BART
    inputs = hf_tokenizer2.batch_encode_plus([v.answer_text_join], max_length=1024, return_tensors='pt')
    
    # generate summary
    gen_text = hf_model2.generate(inputs['input_ids'].to('cuda:1'), 
                                 max_length=130, min_length=30,
                                 early_stopping=True, num_beams=4, num_return_sequences=3)
    
    outputs += [ hf_tokenizer2.decode(txt, skip_special_tokens=True, clean_up_tokenization_spaces=False)
               for txt in gen_text ]
    
    summarization_res.append(outputs)
    
print(len(summarization_res))

371
CPU times: user 11min, sys: 35.6 s, total: 11min 35s
Wall time: 11min 36s


Add preds to summary df

In [35]:
cols = [
    'pred_summary_cnndm_1', 'pred_summary_cnndm_2', 'pred_summary_cnndm_3', 
    'pred_summary_xsum_1', 'pred_summary_xsum_2', 'pred_summary_xsum_3'
]
preds_df = pd.DataFrame(summarization_res, columns=cols)

In [36]:
summarization_df = pd.concat([summarization_df, preds_df], axis=1)

In [37]:
summarization_df.head(20)

,index,benchmark_survey_type,client_id,question_ans_id,question_report_abbr,answer_text_join,answer_text_count,question_category_abbr,benchmark_level1,benchmark_level2,benchmark_level3,group_level1_code,group_level2_code,group_level3_code,group_level4_code,group_level5_code,group_level6_code,group_level7_code,group_level8_code,custom_value1,custom_value2,custom_value3,custom_value4,custom_value5,custom_value6,custom_value7,custom_value8,custom_value9,custom_value10,custom_value11,custom_value12,custom_value13,custom_value14,custom_value15,benchmark_identity,benchmark_sub_identity,client_identity,client_sub_identity,pred_summary_1,pred_summary_2,pred_summary_3,pred_summary_cnndm_1,pred_summary_cnndm_2,pred_summary_cnndm_3,pred_summary_xsum_1,pred_summary_xsum_2,pred_summary_xsum_3
0,0,SAW,UCANR,93005,Conduct & Behavioral - Comments,Is it possible the commitment to diversity is so strong that it goes so far in one direction tha...,136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I feel harassed and bullied by an employee out of my department and don't feel like there is any...,"I would like to point out that I would say ""Never"" for all areas, except for a single individual...",I feel harassed and bullied by an employee out of my department and don't feel like there is any...,I feel harassed and bullied by an employee out of my department and don't feel like there is any...,"I would like to point out that I would say ""Never"" for all areas, except for a single individual...",I feel harassed and bullied by an employee out of my department and don't feel like there is any...,"As part of a series of responses to a survey on diversity at the University of California, Berke...","As part of a series of responses to a survey on diversity at the University of California, Berke...","As part of a series of responses to a survey on diversity at the University of California, Berke..."
1,1,SAW,UCANR,93069,Comments re Work Environment at UC ANR,I believe ANR leadership is completely disconnected from the work of campus-based specialists. W...,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I'm frustrated with wages and lack of opportunity to move up in my job classification. I believe...,I'm frustrated with wages and lack of opportunity to move up in my job classification. I believe...,I believe ANR leadership is completely disconnected from the work of campus-based specialists. I...,I'm frustrated with wages and lack of opportunity to move up in my job classification. I believe...,I'm frustrated with wages and lack of opportunity to move up in my job classification. I believe...,I believe ANR leadership is completely disconnected from the work of campus-based specialists. I...,"As part of a series of responses to a report by the University of California, Davis, on the stat...","As part of a series of responses to a report by the University of California, Davis, on the stat...","As part of a series of responses to a report by the University of California, Davis, on the stat..."
2,2,SAW,UCANR,93005,Conduct & Behavioral - Comments,Is it possible the commitment to diversity is so strong that it goes so far in one direction tha...,136,EDI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I feel harassed and bullied by an employee out of my department and don't feel like there is any...,"I would like to point out that I would say ""Never"" for all areas, except for a single individual...",I feel harassed and bullied by an employee out of my department and don't feel like there is any...,I feel harassed and bullied by an employee out of my department and don't feel like there is any...,"I would like to point out that I would say ""Never"" for all areas, except for a single individual...",I feel harassed and bullied by an 

In [38]:
len(summarization_df)

371

In [39]:
summarization_df.to_excel('./data/anr_summaries.xlsx', index=False)

## Data

Basic Configuration

In [ ]:
vocab = pickle.load(open(LM_PATH/'vocab.pkl', 'rb')); len(vocab)

In [ ]:
bsz = 80
bptt= 72
wd = 1e-7

In [ ]:
chunksize = 24000

include_fld_tok = True
include_bos_tok = True
include_eos_tok = False

In [ ]:
# define what text columns to use (can be multiple)
corpus_cols = ['theme', 'answer_text'] 

# define how to identify the text we are using for the LM
corpus_suf = '_multitask' #'_cleaned'

Prepare the data source

In [ ]:
train_df = pd.read_csv(STANDARD_THEME_META_PATH/'train.csv')
valid_df = pd.read_csv(STANDARD_THEME_META_PATH/'test.csv')

In [ ]:
train_df.is_example.dtype

Remove any rows whre the "corpus_cols" are nan

In [ ]:
train_df.dropna(subset=corpus_cols, inplace=True)
valid_df.dropna(subset=corpus_cols, inplace=True)

In [ ]:
# list(set(train_df.theme.unique()))

In [ ]:
# vocab.itos += [ item.lower() for item in list(set(train_df.theme.unique())) ]
# vocab.stoi = collections.defaultdict(int,{v:k for k,v in enumerate(vocab.itos)})

In [ ]:
train_df['is_valid'] = False
valid_df['is_valid'] = True
df = pd.concat([train_df, valid_df])

In [ ]:
# prepend custom tokenization rules to defaults
custom_lowercase = partial(lowercase, add_bos=include_bos_tok, add_eos=include_eos_tok)

custom_tok_rules = defaults.text_proc_rules[:-1] + [custom_lowercase, 
                                                    make_replacements, 
                                                    fix_ampm, 
                                                    fix_sentence_ends, 
                                                    fix_hyphenated_words]

Using the mid-level `DataBlocks` API

In [ ]:
%%time

blocks = (
    TextBlock.from_df(corpus_cols, vocab=vocab, seq_len=bptt, rules=custom_tok_rules, mark_fields=include_fld_tok),
    RegressionBlock(),
    CategoryBlock()
)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('text'),
                   get_y=[ColReader('avg_sentiment'), ColReader('is_example')],
                   splitter=ColSplitter(col='is_valid'), 
                   n_inp=1)

In [ ]:
# dblock.summary(df)

In [ ]:
%%time
dls = dblock.dataloaders(df, bs=bsz, seq_len=bptt)

In [ ]:
print((
    f'The Datasets vocab has {len(dls.vocab)} items : the inputs vocab ({len(dls.vocab[0])} items), '
    f'and the targets ({len(dls.vocab[1])} items)'))

In [ ]:
print(dls.train_ds[21])

In [ ]:
dls.show_batch(max_n=2)

In [ ]:
batch = next(iter(dls.train))
print(batch[0].size(), batch[0].type(), batch[1].size(), batch[2].size(), batch[1].type(), batch[2].type(), bsz)

In [ ]:
' '.join([ dls.vocab[0][idx] for idx in batch[0][0,:] ])

Save the `Dataloaders` object for future use

In [ ]:
# save dataloaders
torch.save(dls, STANDARD_THEME_META_PATH/f'data_standard_theme_meta.pkl')
# dls = torch.load(STANDARD_THEME_META_PATH/f'data_standard_theme_meta.pkl')

## Training

Configure a forward or backwards run

In [ ]:
backwards = True
m_suf = '_multitask' #'_cleaned'
m_pre = 'bwd_' if (backwards) else 'fwd_'

dls = torch.load(STANDARD_THEME_META_PATH/f'data_standard_theme_meta.pkl')

def reverse_text(nums): return nums.flip(0)
if (backwards): 
    dls.tfms.append(Transform(reverse_text))
    dls.before_batch = partial(pad_input_chunk, pad_first=not backwards)

In [ ]:
# copied from /lm/models -> class/models (both fwd and bwd weights)
! cp {LM_PATH/'models/*_lm_enc.pth'} {STANDARD_THEME_META_PATH/'models/'}  

Configure MM head

In [ ]:
class MM(Module):
    def __init__(self, in_features=50): 
        super().__init__()
        self.pred_is_example = nn.Linear(in_features, 2, bias=False)
        self.pred_avg_sentiment = nn.Linear(in_features, 1, bias=False)
        self.pred_avg_sent_range = SigmoidRange(1., 5.1)
        
    def forward(self, x):
        is_example = self.pred_is_example(x)
        avg_sentiment = self.pred_avg_sent_range(self.pred_avg_sentiment(x))
        
        return avg_sentiment, is_example

In [ ]:
is_example_weights = list(np.max(train_df.is_example.value_counts()) /train_df.is_example.value_counts())
print(is_example_weights)

Configure learner

In [ ]:
# define metrics
def sentiment_mse(preds, *targs):
    return mse(preds[0], targs[0])

def is_example_acc(preds, *targs):
    return accuracy(preds[1], targs[1])

In [ ]:
# define callbacks
best_model_cb = SaveModelCallback(monitor='valid_loss', comp=np.less, fname=f'{m_pre}mm_bestmodel{m_suf}')

In [ ]:
# define our custom multi-target loss
multi_targ_loss = MultiTargetLoss(loss_classes=[MSELossFlat, CrossEntropyLossFlat],
                                  loss_classes_kwargs=[{}, {'weight': FloatTensor(is_example_weights).to('cuda:1')}],
                                  weights=[1, 0.1], 
                                  reduction='mean')

In [ ]:
learn_cbs = [best_model_cb]
fit_cbs = []

learn_metrics = [is_example_acc, sentiment_mse]

# build learner
learn = text_classifier_learner(dls, 
                                AWD_LSTM, 
                                pretrained=False,
                                alpha=2.0, beta=1.0,      # default - alpha=2.0, beta=1.0
                                moms=(0.8,0.7,0.8),       # default - (0.95, 0.85, 0.95)
                                wd=wd,                    # default - None
                                seq_len=bptt,             # default - 72
                                drop_mult=0.5,            # default - 0.5
                                lin_ftrs=[50],            # default - [50]
                                ps=[0.1],                 # default - [0.1]
                                metrics=learn_metrics, 
                                cbs=learn_cbs,
                                n_out=1,
                                path=STANDARD_THEME_META_PATH)

learn.loss_func = multi_targ_loss

In [ ]:
# learn.summary()

In [ ]:
learn = learn.load_encoder(f'{m_pre}lm_enc')

In [ ]:
learn.model._modules['1']._modules['layers']._modules['1']

In [ ]:
learn.model._modules['1']._modules['layers']._modules['1']._modules['2'] = MM(50)

In [ ]:
learn.model = learn.model.to('cuda:1')

In [ ]:
learn.splitter = awd_lstm_clas_split
learn.opt = learn.create_opt()

In [ ]:
learn.freeze()

In [ ]:
learn.summary()

In [ ]:
# learn.show_training_loop()

In [ ]:
best_model_path = STANDARD_THEME_META_PATH/f'models/{m_pre}mm_bestmodel{m_suf}*'
if (best_model_path.exists()): best_model_path.unlink(missing_ok=False)

Train

In [ ]:
lr= 5e-2

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, lr_max=lr, cbs=fit_cbs)

In [ ]:
learn.save(f'{m_pre}mm_last_ft{m_suf}')

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, lr_max=slice(5e-2/(2.6**4), 5e-2), cbs=fit_cbs)

In [ ]:
learn.save(f'{m_pre}mm_last2_ft{m_suf}')

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, lr_max=slice(1e-2/(2.6**4),5e-3), cbs=fit_cbs)

In [ ]:
learn.save(f'{m_pre}mm_last3_ft{m_suf}')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr_max=slice(5e-3/(2.6**4),5e-3), cbs=fit_cbs)

In [ ]:
learn.save(f'{m_pre}mm{m_suf}')

Export model for inference

In [ ]:
learn.export(fname=f'{m_pre}export_mm{m_suf}.pkl')

Look at results

In [ ]:
learn = learn.load(f'{m_pre}mm_bestmodel{m_suf}')

In [ ]:
learn.validate() # ... returns [loss, metrics]

In [ ]:
learn.show_results(max_n=4, trunc_at=60)

Use it for inference

In [ ]:
inf_learn = load_learner(STANDARD_THEME_META_PATH/f'{m_pre}export_mm{m_suf}.pkl')

In [ ]:
inf_learn.predict('I feel very uncomfortable talking to my supervisor even though she loves dogs. Help!!!', 
                  with_input=True)

Review final validation loss for best model

In [ ]:
print(STANDARD_THEME_META_LABELS)

In [ ]:
learn = learn.load(f'{m_pre}mm_bestmodel{m_suf}')

In [ ]:
learn.validate()

In [ ]:
probs, targs, loss = learn.get_preds(dl=dls.valid, with_loss=True)

print(f'Validation Loss: {loss.mean()}')
# print(f'Validation Loss (per label): {loss.mean(dim=0)}') # ... no longer works (see forum comment from sylvain)

In [ ]:
for prob_sent, prob_example, targ_sent, targ_example in zip(*probs,*targs):
    print(prob_sent, targ_sent, torch.argmax(prob_example, dim=-1), targ_example)

In [ ]:
learn.predict("No one cares. No one listens.  My supervisor doesn't follow ethical conduct.")

In [ ]:
learn.predict("plenty of parking available. found a spot every day I came to work. availability is solid")

## Review

### Lets look at validation metrics

In [ ]:
# predictions for a single model using the learner's model and data loaders
learn = learn.load(f'{m_pre}mm_bestmodel{m_suf}')
learn.model.cuda(1)
probs, targs, loss  = learn.get_preds(with_loss=True)

probs[0].shape, probs[1].shape, targs[0].shape, targs[1].shape, loss.mean()

In [ ]:
is_example_prob_true = torch.softmax(probs[1], dim=-1)[:,1]
# is_example_prob_true

In [ ]:
# determine optimal threshold based on desired f-score
f05 = OptimalMultiThresholdMetrics(beta=0.5, start=0.05, end=.9, sigmoid=False, 
                                   average='binary', sample_weight=None)
f1 = OptimalMultiThresholdMetrics(beta=1, start=0.05, end=.9, sigmoid=False, 
                                   average='binary', sample_weight=None)
f2 = OptimalMultiThresholdMetrics(beta=2, start=0.05, end=.9, sigmoid=False, 
                                   average='binary', sample_weight=None)

In [ ]:
is_example_threshold_f05 = f05.opt_th(is_example_prob_true, targs[1])
is_example_threshold_f1 = f1.opt_th(is_example_prob_true, targs[1])
is_example_threshold_f2 = f2.opt_th(is_example_prob_true, targs[1])

is_example_threshold_f05, is_example_threshold_f1, is_example_threshold_f2

In [ ]:
res = skm.fbeta_score(targs[1], (is_example_prob_true > is_example_threshold_f1), beta=1, average='binary')
res

In [ ]:
preds = ((is_example_prob_true > is_example_threshold_f1).byte() == targs[1].byte()).float().mean()
preds.item()

In [ ]:
# determine accuracy based on optimal threshold
is_example_val_acc_f05 = accuracy_multi(torch.argmax(probs[1], dim=-1), targs[1], is_example_threshold_f05, sigmoid=False).item()
is_example_val_acc_f1 = accuracy_multi(torch.argmax(probs[1], dim=-1), targs[1], is_example_threshold_f1, sigmoid=False).item()
is_example_val_acc_f2 = accuracy_multi(torch.argmax(probs[1], dim=-1), targs[1], is_example_threshold_f2, sigmoid=False).item()

is_example_val_acc_f05, is_example_val_acc_f1, is_example_val_acc_f2

### Review classifier - Is Example

In [ ]:
eval_targs = targs[1].flatten() # targs[:,0]
eval_probs = is_example_prob_true.flatten()

In [ ]:
eval_targs.shape, eval_probs.shape

#### Classification Accuracy

The percentage of correct predictions.  Answers the question, *"Overall, how often is the classifier correct?"*

In [ ]:
print(skm.accuracy_score(eval_targs, (eval_probs > is_example_threshold_f1).float(), sample_weight=None))

#### Null Accuracy
 
The accuracy achieved by always predicting the most frequent class.  Answers the question, *"What would the accuracy be by always predicting the most frequent case?"*

In [ ]:
u_classes, u_counts = np.unique(eval_targs, return_counts=True)
most_freq_class, most_freq_class_count = u_classes[np.argmax(u_counts)], np.max(u_counts)
print(most_freq_class, most_freq_class_count)

In [ ]:
most_freq_class_count / len(eval_targs)

#### Cohen's kappa

This measure is intended to compare labelings by different human annotators (not a classifier vs. ground truth)

Kappa socres are between -1 and 1 ( >= .8 is generally considered good agreement; <= 0 means no agreement ... e.g., practically random labels)

In [ ]:
print(skm.cohen_kappa_score(eval_targs, (eval_probs > is_example_threshold_f1).float(), 
                            weights=None, sample_weight=None))

#### Confusion Matrix

Describes the performance of a classification model

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, 
                          title='Confusion matrix', cmap=plt.cm.Blues, print_info=False):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        if (print_info): print("Normalized confusion matrix")
    else:
        if (print_info): print('Confusion matrix, without normalization')

    if (print_info): print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar(fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(None)

In [ ]:
cm = skm.confusion_matrix(eval_targs, (eval_probs > is_example_threshold_f1).float(), sample_weight=None)

In [ ]:
# Plot non-normalized confusion matrix
fig = plt.figure(figsize=(12,8))
plt.subplot(1, 2, 1)
plot_confusion_matrix(cm, classes=u_classes,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.subplot(1, 2, 2)
plot_confusion_matrix(cm, classes=u_classes, normalize=True,
                      title='Normalized confusion matrix')

fig.subplots_adjust(wspace=0.5)
plt.show()

In [ ]:
print(skm.classification_report(eval_targs, (eval_probs > is_example_threshold_f1).float(), [0,1], 
                                sample_weight=None))

#### Raw probability distribution

Useful to see how the threshold can be adjusted to increase sensitivity or specificity

In [ ]:
plt.hist(eval_probs, bins=10)
# plt.xlim(0,1)
plt.title('Histogram of predicted probabilities')
plt.xlabel('Predicted probability of IsVeryPositive')
plt.ylabel('Frequency')

#### ROC curves and Area Under the Curve (AUC)

***ROC Curve*** answers the question, *"How would sensitivity and specificity be affected by various thresholds without changing the threshold?"*  It is a way **to visualize the performance of a binary classifier.**

The ROC curve can help you **choose a threshold** that balances sensitivity and specificity based on your particular business case.

ROC curves visualize all possible classification thresholds whereas misclassification rate only represents your error rate for a single threshold.

A classifier that does a good job at separating the classes will have a ROC curve that hugs the upper left corner of the plot.  Converseley, a classifier the does a poor job separating the classes will have a ROC curve that is close to the diagonal line (0,0 -> 1,1).  That diagonal line represents a classifier that does no better than random guessing.

In [ ]:
fpr, tpr, thresholds = skm.roc_curve(eval_targs, eval_probs, sample_weight=None)

In [ ]:
plt.plot(fpr, tpr)
plt.xlim = ([0.0, 1.0])
plt.ylim = ([0.0, 1.0])
plt.title('ROC curve for all labels')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity/Recall)')
plt.grid(True)

***AUC*** = the percentage of the ROC plot that is underneath the curve.  

AUC summarizes the performance of a classifier in a **single number**.  It says, *"If you randomly chose one positive and one negative observation, what is the likelihood that your classifier will assign a higher predicted probability to the positive observation."*

**An AUC of ~ 0.8 is very good while an AUC of ~ 0.5 represents a poor classifier.**

The ROC curve and AUC are insensitive to whether your predicted probabilities are properly calibrated to actually represent probabilities of class membership (e.g., it works if predicted probs range from 0.9 to 1 instead of 0 to 1).  All the AUC metric cares about is how well your classifier separated the two classes

Notes:
1.  AUC is useful even when there is **high class imbalance** (unlike classification accuracy)
2.  AUC is useful even when predicted probabilities are not properly calibrated (e.g., not between 0 and 1)

In [ ]:
print(skm.roc_auc_score(eval_targs, eval_probs, average='weighted', sample_weight=None))

### Review regression - Sentiment

In [ ]:
# mse
skm.mean_squared_error(targs[0], probs[0])

In [ ]:
# rmse
math.sqrt(skm.mean_squared_error(targs[0], probs[0]))

In [ ]:
# mae
skm.mean_absolute_error(targs[0], probs[0])

## Ensemble forwards and backwards passes

In [ ]:
try:
    learn_fwd.purge(); learn = None;
    learn_bwd.purge(); learn_bwd = None;
    gc.collect()
    torch.cuda.empty_cache()
except: pass

bsz = 80
m_suf = '_multitask'

dls = torch.load(STANDARD_THEME_META_PATH/f'data_standard_theme_meta.pkl')

learn_fwd = load_learner(fname=STANDARD_THEME_META_PATH/f'fwd_export_mm{m_suf}.pkl')
learn_fwd.dls = dls
learn_bwd = load_learner(fname=STANDARD_THEME_META_PATH/f'bwd_export_mm{m_suf}.pkl')
learn_bwd.dls = dls

In [ ]:
learn_fwd.loss_func = multi_targ_loss
learn_bwd.loss_func = multi_targ_loss

In [ ]:
probs_fwd, lbl_fwd, loss_fwd = learn_fwd.get_preds(with_loss=True, reorder=True)
probs_bwd, lbl_bwd, loss_bwd = learn_bwd.get_preds(with_loss=True, reorder=True)

probs_fwd[0].shape, probs_bwd[0].shape, loss_fwd.shape

In [ ]:
loss_fwd.mean(), loss_bwd.mean(), (loss_fwd.mean() + loss_bwd.mean()) / 2

In [ ]:
probs_final_sent = (probs_fwd[0] + probs_bwd[0]) / 2
probs_final_is_example = (probs_fwd[1] + probs_bwd[1]) / 2

### Results

In [ ]:
# determine optimal threshold based on desired f-score
f05 = OptimalMultiThresholdMetrics(beta=0.5, start=0.05, end=.9, sigmoid=False, 
                                   average='binary', sample_weight=None)
f1 = OptimalMultiThresholdMetrics(beta=1, start=0.05, end=.9, sigmoid=False, 
                                   average='binary', sample_weight=None)
f2 = OptimalMultiThresholdMetrics(beta=2, start=0.05, end=.9, sigmoid=False, 
                                   average='binary', sample_weight=None)

In [ ]:
is_example_prob_true_fwd = torch.softmax(probs_fwd[1], dim=-1)[:,1]
is_example_prob_true_bwd = torch.softmax(probs_bwd[1], dim=-1)[:,1]
is_example_prob_true_ensemble = torch.softmax(probs_final_is_example, dim=-1)[:,1]

# is_example_prob_true_fwd, is_example_prob_true_bwd, is_example_prob_true_ensemble

In [ ]:
# determine optimal threshold based on desired f-score
is_example_threshold_f05 = f05.opt_th(is_example_prob_true_fwd, lbl_fwd[1])
is_example_threshold_f1 = f1.opt_th(is_example_prob_true_fwd, lbl_fwd[1])
is_example_threshold_f2 = f2.opt_th(is_example_prob_true_fwd, lbl_fwd[1])

is_example_threshold_f05, is_example_threshold_f1, is_example_threshold_f2

# determine accuracy based on optimal threshold
val_acc_f05 = accuracy_multi(is_example_prob_true_fwd, lbl_fwd[1], is_example_threshold_f05, sigmoid=False).item()
val_acc_f1 = accuracy_multi(is_example_prob_true_fwd, lbl_fwd[1], is_example_threshold_f1, sigmoid=False).item()
val_acc_f2 = accuracy_multi(is_example_prob_true_fwd, lbl_fwd[1], is_example_threshold_f2, sigmoid=False).item()

print('Fowards Only\n-------------')
print(f'f05:\tOptimal threshold = {is_example_threshold_f05}\t(Accuracy = {val_acc_f05})')
print(f'f1:\tOptimal threshold = {is_example_threshold_f1}\t(Accuracy = {val_acc_f1})')
print(f'f2:\tOptimal threshold = {is_example_threshold_f2}\t(Accuracy = {val_acc_f2})')

print(f'\nAccuracy: {skm.accuracy_score(lbl_fwd[1], (is_example_prob_true_fwd > is_example_threshold_f1).float())}')

In [ ]:
# determine optimal threshold based on desired f-score
is_example_threshold_f05 = f05.opt_th(is_example_prob_true_bwd, lbl_fwd[1])
is_example_threshold_f1 = f1.opt_th(is_example_prob_true_bwd, lbl_fwd[1])
is_example_threshold_f2 = f2.opt_th(is_example_prob_true_bwd, lbl_fwd[1])

is_example_threshold_f05, is_example_threshold_f1, is_example_threshold_f2

# determine accuracy based on optimal threshold
val_acc_f05 = accuracy_multi(is_example_prob_true_bwd, lbl_fwd[1], is_example_threshold_f05, sigmoid=False).item()
val_acc_f1 = accuracy_multi(is_example_prob_true_bwd, lbl_fwd[1], is_example_threshold_f1, sigmoid=False).item()
val_acc_f2 = accuracy_multi(is_example_prob_true_bwd, lbl_fwd[1], is_example_threshold_f2, sigmoid=False).item()

print('Backwards Only\n-------------')
print(f'f05:\tOptimal threshold = {is_example_threshold_f05}\t(Accuracy = {val_acc_f05})')
print(f'f1:\tOptimal threshold = {is_example_threshold_f1}\t(Accuracy = {val_acc_f1})')
print(f'f2:\tOptimal threshold = {is_example_threshold_f2}\t(Accuracy = {val_acc_f2})')

print(f'\nAccuracy: {skm.accuracy_score(lbl_fwd[1], (is_example_prob_true_bwd > is_example_threshold_f1).float())}')

In [ ]:
# determine optimal threshold based on desired f-score
is_example_threshold_f05 = f05.opt_th(is_example_prob_true_ensemble, lbl_fwd[1])
is_example_threshold_f1 = f1.opt_th(is_example_prob_true_ensemble, lbl_fwd[1])
is_example_threshold_f2 = f2.opt_th(is_example_prob_true_ensemble, lbl_fwd[1])

is_example_threshold_f05, is_example_threshold_f1, is_example_threshold_f2

# determine accuracy based on optimal threshold
val_acc_f05 = accuracy_multi(is_example_prob_true_ensemble, lbl_fwd[1], is_example_threshold_f05, sigmoid=False).item()
val_acc_f1 = accuracy_multi(is_example_prob_true_ensemble, lbl_fwd[1], is_example_threshold_f1, sigmoid=False).item()
val_acc_f2 = accuracy_multi(is_example_prob_true_ensemble, lbl_fwd[1], is_example_threshold_f2, sigmoid=False).item()

print('Ensemble Only\n-------------')
print(f'f05:\tOptimal threshold = {is_example_threshold_f05}\t(Accuracy = {val_acc_f05})')
print(f'f1:\tOptimal threshold = {is_example_threshold_f1}\t(Accuracy = {val_acc_f1})')
print(f'f2:\tOptimal threshold = {is_example_threshold_f2}\t(Accuracy = {val_acc_f2})')

print(f'\nAccuracy: {skm.accuracy_score(lbl_fwd[1], (is_example_prob_true_ensemble > is_example_threshold_f1).float())}')

In [ ]:
# sentiment metrics
sentiment_mae = skm.mean_absolute_error(lbl_fwd[0], probs_final_sent)
sentiment_mse = skm.mean_squared_error(lbl_fwd[0], probs_final_sent)
sentiment_rmse = math.sqrt(skm.mean_squared_error(lbl_fwd[0], probs_final_sent))

print(sentiment_mae, sentiment_mse, sentiment_rmse)

In [ ]:
final_valid_loss = (loss_fwd.mean() + loss_bwd.mean()) / 2; final_valid_loss

In [ ]:
float(sentiment_mae)

In [ ]:
# save this information to be added to inference returned data
model_results = {
    'is_example_threshold_f05': is_example_threshold_f05,
    'is_example_threshold_f1': is_example_threshold_f1,
    'is_example_threshold_f2': is_example_threshold_f2,
    
    'val_acc_f05': val_acc_f05,
    'val_acc_f1': val_acc_f1,
    'val_acc_f2': val_acc_f2,
    
    'sentiment_mae': float(sentiment_mae),
    'sentiment_mse': float(sentiment_mse),
    'sentiment_rmse': float(sentiment_rmse),
    
    'final_valid_loss': final_valid_loss.item()
}

with open(STANDARD_THEME_META_PATH/'model_results.json', 'w') as f: json.dump(model_results, f)

### Inference (ad-hoc documents)

In [ ]:
print(STANDARD_THEME_META_LABELS)

In [ ]:
test_comments = [
    'The parking situation REALLY sucks around here.  It needs to be fixed',
    'I LOVE working at UCSD!!!  It is wonderful',
    """Some staff are just uninformed.There is no support for solo-individual study (no closed off rooms).
        Once a guy (quite tall) walked in into the girl's restroom and used the stalls standing up. 
        There was no line in the guy's restroom. This happened when I done and was going to walk out. 
        I was extremely uncomfortable""",
    "I love UCSD!!! It is a terrible place to work!",
    "I was really uncomfortable to express my opinion!!!"
]

for c in test_comments: print(learn_fwd.predict(c, with_input=True))

### Inference (batch ensemble)

In [ ]:
import datetime
yyyymmdd = datetime.date.today().strftime("%Y%m%d")

m_suf = '_multitask'

# device = torch.device('cpu')
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
verbatims_df = pd.read_csv(STANDARD_THEME_SAW_PATH/'20200511_ensemble_predictions_multilabel.csv', 
                           dtype={**TASK_LM_DTYPES}, parse_dates=[])

inf_df = verbatims_df.copy() #verbatims_df[test_df.SurveyID == 130].copy()
inf_df.reset_index(drop=True, inplace=True)
print(len(verbatims_df))

corpus_cols = ['theme', 'AnswerText'] 

In [ ]:
inf_df.head(1)

In [ ]:
pred_theme_cols = filter_col = [col for col in inf_df if col.startswith('prob_')]
# pred_theme_cols

In [ ]:
inf_df = inf_df.melt(id_vars=list(TASK_LM_DTYPES.keys()) + ['threshold_f05', 'threshold_f1', 'threshold_f2', 'val_acc_f05', 'val_acc_f1', 'val_acc_f2', 'val_loss'], 
                     value_vars=pred_theme_cols, 
                     var_name='theme', 
                     value_name='theme_prob')
len(inf_df)

In [ ]:
inf_df = inf_df.loc[inf_df.theme_prob >= inf_df.threshold_f2]
len(inf_df)

In [ ]:
inf_df['url_friendly_theme'] = inf_df.theme.apply(
    lambda s: re.sub("(.*?)_([a-zA-Z])","\g<1> \g<2>",s).replace('prob', '').strip().title().replace(' ',''))

inf_df['theme'] = inf_df.url_friendly_theme.apply(lambda s: re.sub("([a-z])([A-Z])","\g<1> \g<2>",s))

In [ ]:
inf_df.reset_index(inplace=True)

In [ ]:
inf_df.head()

In [ ]:
def get_model_results(backwards:bool=False, m_suf:str='multilabel'):
    
    model_prefix = 'bwd' if backwards else 'fwd'
    
    # 1. grab learner, procs, and data
    inf_learn = load_learner(fname=STANDARD_THEME_META_PATH/f'{model_prefix}_export_mm_{m_suf}.pkl', cpu=False)
    inf_learn.model = inf_learn.model.to(device)
    inf_learn.model = inf_learn.model.eval()
    
    # 2. define a suitable dataloader
    tok_inf_df, tok_counts = tokenize_df(inf_df, corpus_cols)
    inf_dl = inf_learn.dls.test_dl(tok_inf_df, rm_type_tfms=None, bs=128)
    if (backwards): inf_dl.tfms.add(Transform(lambda nums: nums.flip(0)))

    # 3. get probs
    test_probs_sent, test_probs_is_example = [], []
    with torch.no_grad():
        for index, b in enumerate(inf_dl):
            if index % 1000 == 0:  print(index)

            # reset hidden state (if you don't do this you will OOM)
            inf_learn.model.reset()
            
            # note: even though there is no targets, each batch is a tuple!
            probs, raw_outputs, outputs = inf_learn.model(b[0])
            
            # why "detach"? the computation of gradients wrt the weights of netG can be fully 
            # avoided in the backward pass if the graph is detached where it is.
            test_probs_sent.append(to_detach(probs[0]))
            test_probs_is_example.append(to_detach(probs[1]))

    all_probs_sent = L(torch.cat(test_probs_sent))
    all_probs_is_example = L(torch.cat(test_probs_is_example))

    # 4. ensure results are returned in order
    # test_dl.get_idxs() => unsorted/original order items
    all_probs_sent = all_probs_sent[0][np.argsort(inf_dl.get_idxs())]
    all_probs_is_example = all_probs_is_example[0][np.argsort(inf_dl.get_idxs())]

    # 5. return ordered results
    inf_learn, inf_data = None, None; gc.collect()
    
    return all_probs_sent, all_probs_is_example

In [ ]:
%time

probs_fwd_sent, probs_fwd_is_example = get_model_results(backwards=False, m_suf='multitask')
probs_bwd_sent, probs_bwd_is_example = get_model_results(backwards=True, m_suf='multitask')

probs_final_sent = (probs_fwd_sent + probs_bwd_sent) / 2
probs_final_is_example = torch.softmax((probs_fwd_is_example + probs_bwd_is_example) / 2, dim=-1)[:,1]

# probs_final = torch.sigmoid((probs_fwd + probs_bwd) / 2)

print(probs_final_sent.shape)
print(probs_final_sent[:5])
print(probs_final_is_example.shape)
print(probs_final_is_example[:5])

Add the probabilities of each label to `inf_df`

In [ ]:
np.concatenate((probs_final_sent.numpy(), probs_final_is_example.numpy()[:,None]), axis=1)

In [ ]:
combined_probs = np.concatenate((probs_final_sent.numpy(), probs_final_is_example.numpy()[:,None]), axis=1)

In [ ]:
prob_labels = ['prob_' + lbl for lbl in STANDARD_THEME_META_LABELS]
probs_df = pd.DataFrame(combined_probs, columns=prob_labels)
probs_df.head()

In [ ]:
# test_df_filtered.update(probs_df)
final_df = pd.concat([inf_df, probs_df], axis=1)
final_df.head()

Add in predictions based on f1 threshold

In [ ]:
for lbl in STANDARD_THEME_META_LABELS[1:]:
    final_df[f'pred_{lbl}'] = (final_df[f'prob_{lbl}'] > is_example_threshold_f1).astype(np.int64)

Include found thresholds

In [ ]:
final_df['is_example_threshold_f05'] = is_example_threshold_f05
final_df['is_example_threshold_f1'] = is_example_threshold_f1
final_df['is_example_threshold_f2'] = is_example_threshold_f2

final_df['is_example_val_acc_f05'] = val_acc_f05
final_df['is_example_val_acc_f1'] = val_acc_f1
final_df['is_example_val_acc_f2'] = val_acc_f2

In [ ]:
final_df['sentiment_mae'] = sentiment_mae.item()
final_df['sentiment_mse'] = sentiment_mse.item()
final_df['sentiment_rmse'] = sentiment_rmse

In [ ]:
final_df['val_loss_metadata'] = final_valid_loss.item()

In [ ]:
final_df.head()

In [ ]:
import datetime
final_df.to_csv(STANDARD_THEME_META_PATH/f'{yyyymmdd}_ensemble_predictions{m_suf}.csv', index=False)

### Playground

In [ ]:
final_df.head()

In [ ]:
print(final_df[final_df.Id == 589305].AnswerText.values[0])
final_df[final_df.Id == 589305]

In [ ]:
print(final_df.iloc[0].is_example_threshold_f05)
print(final_df.iloc[0].is_example_threshold_f1)
print(final_df.iloc[0].is_example_threshold_f2)
print(final_df.prob_avg_sentiment.mean())

In [ ]:
len(learn.opt.param_groups)

In [ ]:
[ print(f'{lg}\n') for lg in learn.opt.param_groups ]

In [ ]:
losses = np.array([1,2,3,4,5,6,7,8,9,10]); losses

In [ ]:
losses.mean()

In [ ]:
(losses[:5].mean() + losses[5:].mean()) / 2